In [2]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
from bs4 import BeautifulSoup
cwd = subprocess.os.getcwd()

Error importing optional module geopandas
Traceback (most recent call last):
  File "c:\Users\engin\anaconda3\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\engin\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\engin\anaconda3\lib\site-packages\geopandas\__init__.py", line 3, in <module>
    from geopandas.geoseries import GeoSeries  # noqa
  File "c:\Users\engin\anaconda3\lib\site-package

In [3]:
def half_life_to_lambda(half_life):
    '''
    This function takes the e-folding times of the entire decay chain.
    Returns the half-life of the nuclide
    '''
    return half_life / np.log(2)

nuclide_df = pd.read_csv(cwd + '\\NuclideData.csv').iloc[:,1:]
half_lives = nuclide_df['Half life (years)'].to_numpy() 
nuclide_df["e Folding Time (seconds)"] = half_life_to_lambda(half_lives * units.year.to(units.s))
nuclide_df

,Beta-decay energy (keV),Half life (years),Isotope,Beta-decay fraction,Average beta decay energy,e Folding Time (seconds)
0,*,4.182828e-10,63Se,0.0,5.69,1.904357e-02
1,21661.2131,2.725176e-30,5H,0.0,5.69,1.240718e-22
2,-217.0,3.580754e-07,272Bh,0.0,5.69,1.630245e+01
3,-13088.708,7.115560e-07,21Na,0.0,5.69,3.239572e+01
4,*,9.506426e-10,82Mo,0.0,5.69,4.328085e-02
...,...,...,...,...,...,...
3187,9925.3249,2.851928e-08,106Nb,1.0,5.69,1.298426e+00
3188,22940.0,3.168809e-11,59K,0.0,5.69,1.442695e-03
3189,1047.6178,2.564834e-05,146Ce,1.0,5.69,1.167717e+03
3190,-4581.6095,8.156514e-04,73Se,0.0,284.00,3.713497e+04


In [4]:
element_symbols = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 
'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 
'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 
'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 
'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm',
 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 
 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 
 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 
 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og', 'N/A']
len(element_symbols)
#to find the daughter nuclide we only need to increment by 1 in the element symbols.
element_symbols[element_symbols.index('Ni')+1]
#remove numbers from string
s = element_symbols
nuclide_df['Daugher Nucleus'] = [re.sub('\D+', '', n) + s[s.index(re.sub('\d+', '', n))+1] 
if re.sub('\d+', '', n) in s else 'N/A' for n in nuclide_df['Isotope'] ]

In [5]:
def get_isotope_info(isotope, info = None, isotope_column = None,
    dataset = {}, isotope_list = None, lists_to_search = []):                  
  '''
  isotope_list and list_to_search are optional arguments.
  If list_to_search is not provided, then info must be provided.
  If isotope_list is not provided, then dataset and isotope_column
  must be provided.
  '''
  if isotope_list is None:
    isotope_list = list(dataset[isotope_column])
  row = isotope_list.index(isotope)
  if len(lists_to_search) == 0:
    try:
      lists_to_search = list(dataset[info])
    except:
      print("info to search for not entered")
      return
  try: #only works if there are multiple specified lists to search 
    return [target_list[row] for target_list in lists_to_search]
  except:
    return lists_to_search[row]


In [6]:
isotope_list = list(nuclide_df['Isotope'])
lambda_list = list(nuclide_df['e Folding Time (seconds)'])
decay_energy_list = list(nuclide_df['Average beta decay energy'])
daughter_list = list(nuclide_df['Daugher Nucleus'])

In [7]:
def make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns the decay chain.
    '''
    decay_chain = {}
    decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list, 
        lists_to_search = (lambda_list, decay_energy_list, daughter_list))
    while True:
        isotope = decay_chain[isotope][2]
        try:
            decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list,
                lists_to_search = (lambda_list, decay_energy_list, daughter_list))
            if(isotope == decay_chain[isotope][2]):
                return decay_chain
        except:
            return pd.DataFrame(decay_chain, index= ("e-Folding Time (seconds)", 
                                "Average beta-decay energy", "Daughter")).transpose()


In [8]:
#Quickly calculating the decay rate of the i-th generation nuclide
def formulate_decay_rate(e_folding_times):
    '''
    The e-folding times must be in a numpy array.
    Returns a formula for the decay rate of each generation. 
    '''
    exponent_array = -1 / e_folding_times
    decay_rates = len(exponent_array)  * [sym.N(0)]
    decay_rates[0] = sym.exp(t * exponent_array[0]) * exponent_array[0]
    for index, L in enumerate(exponent_array[1:]):
        decay_rates[index+1] = decay_rates[index] * (1 + sym.exp(t * L) * L) 
    return decay_rates

def eval_decay_rates(decay_rates, time_array):
    '''
    This function takes the formula for the decay rate of each generation
    and substitutes each value in the time array for t.
    Each decay_rate must be a sympy expression. 
    https://docs.sympy.org/ 
    Rewrite so that the outer loop is over the time array and the inner loop
    is over the generations. This can be done by using (suggested by copilot: 
    the np.meshgrid function) or a 2d array (my first thought)
    '''
    try: #will only evaluate if decay_rates is an array of sympy expressions
        evaluated_decay_rates = [np.array([formula.subs(t, time) for formula in decay_rates])
                                    for time in time_array]
    except:
        evaluated_decay_rates = [decay_rates.subs(t, time) for time in time_array]
    return evaluated_decay_rates


#Calculating the power density of a decay chain
def calc_power_density(decay_rates, decay_energies, initial_mass):
    '''
    Rewrite such that is sums the entire chain and NOT across time
    decay_rates must be a numpy array in moles/second
    decay_energies must be a numpy array in keV/decay
    This function takes the decay rates and energies of the decay chain.
    Returns the power density in watts/g
    '''
    power_density =  [sum(decay_rate_t * decay_energies) for decay_rate_t in decay_rates]
    '''
    At every different value for time, 
    store the sum of the product of each generation's decay rate and decay energy.
    '''
    power_density = np.array(power_density)
    #convert to W/g
    power_density *=  units.keV.to(units.J) * float(constants.N_A * units.mol) / initial_mass
    return np.abs(power_density.astype(float))


In [9]:
chain = make_decay_chain('32Si', isotope_list, lambda_list, decay_energy_list, daughter_list)
chain

,e-Folding Time (seconds),Average beta-decay energy,Daughter
32Si,7.14789e+09,68.8,32P
32P,1.77861e+06,694.9,32S


In [10]:
decay_rates = formulate_decay_rate(chain['e-Folding Time (seconds)'])
#currently, eval_decay_rates evalautes in < 1 second for 100 time points and 6 generations
time_array = np.logspace(1, 9.5, 10**2)
n_decay_rates = eval_decay_rates(decay_rates, time_array)
n_decay_rates

[array([-1.39901329266888e-10, -1.39901250609852e-10], dtype=object),
 array([-1.39901329224105e-10, -1.39901250567165e-10], dtype=object),
 array([-1.39901329171969e-10, -1.39901250515147e-10], dtype=object),
 array([-1.39901329108436e-10, -1.39901250451758e-10], dtype=object),
 array([-1.39901329031016e-10, -1.39901250374512e-10], dtype=object),
 array([-1.39901328936672e-10, -1.39901250280382e-10], dtype=object),
 array([-1.39901328821705e-10, -1.39901250165675e-10], dtype=object),
 array([-1.39901328681607e-10, -1.39901250025894e-10], dtype=object),
 array([-1.39901328510885e-10, -1.39901249855557e-10], dtype=object),
 array([-1.39901328302845e-10, -1.39901249647987e-10], dtype=object),
 array([-1.39901328049328e-10, -1.39901249395043e-10], dtype=object),
 array([-1.39901327740394e-10, -1.39901249086808e-10], dtype=object),
 array([-1.39901327363930e-10, -1.39901248711194e-10], dtype=object),
 array([-1.39901326905173e-10, -1.39901248253474e-10], dtype=object),
 array([-1.399013263

In [11]:
t_power_densities = calc_power_density(decay_rates = n_decay_rates, decay_energies = np.array(chain['Average beta-decay energy']),
        initial_mass = float(re.sub('\D+', '', chain.index[0])))
px.scatter(y=t_power_densities, x= time_array, log_x=True, log_y=True,
        labels={'x': 'Time (seconds)', 'y': 'Power Density (W/g)'}, title = 'Power Density of ' + chain.index[0] + ' Decay Chain')
#THIS IS MUCH HIGHER THAN WHAT I HAVE IN THE DECADE HALF LIVES POWER DENSITIES ON GITHUB!!!
#Double check decay formulas!!!!

In [12]:
time_array

array([1.00000000e+01, 1.21859274e+01, 1.48496826e+01, 1.80957154e+01,
       2.20513074e+01, 2.68715631e+01, 3.27454916e+01, 3.99034183e+01,
       4.86260158e+01, 5.92553098e+01, 7.22080902e+01, 8.79922544e+01,
       1.07226722e+02, 1.30665705e+02, 1.59228279e+02, 1.94034425e+02,
       2.36448941e+02, 2.88134963e+02, 3.51119173e+02, 4.27871275e+02,
       5.21400829e+02, 6.35375264e+02, 7.74263683e+02, 9.43512101e+02,
       1.14975700e+03, 1.40108553e+03, 1.70735265e+03, 2.08056754e+03,
       2.53536449e+03, 3.08957676e+03, 3.76493581e+03, 4.58792343e+03,
       5.59081018e+03, 6.81292069e+03, 8.30217568e+03, 1.01169710e+04,
       1.23284674e+04, 1.50233808e+04, 1.83073828e+04, 2.23092437e+04,
       2.71858824e+04, 3.31285189e+04, 4.03701726e+04, 4.91947992e+04,
       5.99484250e+04, 7.30527154e+04, 8.90215085e+04, 1.08480964e+05,
       1.32194115e+05, 1.61090788e+05, 1.96304065e+05, 2.39214708e+05,
       2.91505306e+05, 3.55226249e+05, 4.32876128e+05, 5.27499706e+05,
      

In [17]:
all_decay_chains = [make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list) 
                    for isotope in isotope_list]
time_array = np.logspace(1, 9.5, 10)
def calculate_all_power_densities(all_decay_chains, time_array, mean = True):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns power density as a function of time for each decay chain.
    The optional boolean argument mean determines whether the whole time series is returned
    or only its mean. 
    '''
    if mean:
        all_power_densities = [np.mean(calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                )) for chain in all_decay_chains]
    else:
        all_power_densities = [calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                ) for chain in all_decay_chains]
    return all_power_densities

mean_power_densities = calculate_all_power_densities(all_decay_chains[:2], time_array, mean = True)


In [24]:
3.7 * len(all_decay_chains) * units.second.to(units.h) / 2
#it will take 2 hours to run my script in jupyter. I'l convert it to a python script and run it in pypy to see if it's faster.


1.6403333333333336